# Preprocessing with a tokenizer

Comenzamos importando AutoTokenizer de la biblioteca transformers de Hugging Face. El tokenizador se utiliza para convertir texto en un formato que el modelo puede entender, esencial para tareas de procesamiento de lenguaje natural.
Utilizamos el modelo distilbert-base-uncased-finetuned-sst-2-english, que es una versión destilada y afinada del BERT, optimizada para la tarea de análisis de sentimientos en textos en inglés. Este modelo proporciona un equilibrio entre precisión y velocidad, siendo adecuado para inferencias en tiempo real.

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, force_download=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

El proceso de tokenización convierte cada texto en una serie de tokens o identificadores que representan palabras o segmentos de palabras. El modelo utiliza estos tokens para entender y analizar el contenido del texto  
Parámetros   
**padding**=True: Asegura que todas las secuencias tokenizadas tengan la misma longitud rellenando con tokens especiales donde sea necesario.  
**truncation**=True: Recorta las secuencias que excedan la longitud máxima permitida por el modelo.  
**return_tensors**="pt": Devuelve los tokens en un tensor de PyTorch, adecuado para el modelo que estamos utilizando.

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


# Going through the model
Después de preparar los textos mediante tokenización, el siguiente paso es cargar el modelo que usaremos para clasificar el sentimiento de los textos. En esta sección, cargaremos un modelo pre-entrenado de Hugging Face Transformers específicamente afinado para la tarea de clasificación de sentimientos.

In [9]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, force_download=True)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [11]:
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


Una vez que hemos realizado la inferencia utilizando el modelo, es útil entender la estructura de los resultados. En este caso, estamos interesados en los logits, que son las salidas crudas del modelo antes de aplicar una función de activación como la softmax.

¿Qué son los Logits?
Los logits son las puntuaciones sin procesar que cada clase recibe en un problema de clasificación. En nuestro contexto, estos logits representarán las puntuaciones para las categorías de sentimientos (positivo y negativo).

In [13]:
print(outputs.logits.shape)

torch.Size([2, 2])


# Postprocessing the output

In [14]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


Después de obtener los logits del modelo, el siguiente paso es convertir estas puntuaciones brutas en probabilidades utilizando la función softmax. Esto nos ayudará a interpretar más fácilmente los resultados como probabilidades de cada clase.

¿Qué hace la función Softmax?
La función softmax convierte los logits, que son valores reales arbitrarios, en valores que suman 1, lo que los hace interpretables como probabilidades. Cada valor representa la probabilidad de que la entrada pertenezca a una clase particular.

In [15]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [16]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# Using Pipeline
Para simplificar el proceso de clasificación de sentimientos, podemos utilizar la función pipeline de la biblioteca Hugging Face Transformers. Esta herramienta nos permite cargar directamente un modelo con sus dependencias asociadas y aplicarlo a los textos de forma inmediata.

In [18]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]